# **Prototype de simulation du modèle IAA** 

But de ce notebook : établir pour un deal donné les simulations de pertes

**Algorithme** : 

**Etape 1** : Simuler **SG risk factor** + **Conduit risk factor** + **Asset class factor**  :  [CF] p37-p40

$ M_f \overset{d}{=} \mathcal{N}(0,1) $

$ C_f = b M_f + \sqrt{1-b^2} X_c$ avec $X_c \overset{d}{=} \mathcal{N}(0,1)$

$ S_{fAssetClass} = a C_f + \sqrt{1-a^2} X_{AssetClass}$ avec $ X_{AssetClass} \overset{d}{=} \mathcal{N}(0,1) $

**Etape 2** : On simule le **seller/deal factor** : [SF] p 40-p42

$ S_{f_k} = \rho_{ACF} S_{fAssetClass} + \sqrt{1- \rho_{ACF}^2} X_k $ avec $X_k \overset{d}{=} \mathcal{N}(0,1)$

**Etape 3** : On fait une boucle sur chaque mois jusqu'à un nombre T de mois égal au **Commitment maturity** :

Pour chaque mois t, on test les 3 conditions de la **general rule condition** (p45) pour tester si on passe à la phase d'amortissement :

- On simule le **mensual risk factor** $P_{f_k}(t) = S_{f_k}$ (en principe inutile car il est constant toute les mois car égal à $S_{f_k}$) p43
- On tire le nouveau Rating du seller au mois t :

    $R_{seller}(t+1) = NewRating(\sqrt{\rho} S_{f_k} + \sqrt{1 - \rho} \mathcal{N}(0,1) , R_{ini})$ p42 en bas de la page

On test **[1ère condition]** : Défaut du seller (c'est à dire obtenir un rating D)

- On test **[2eme condition]** : $t \le commitment\ maturity$

- Pour chaque $BigDebtor_j$, on tire son nouveau rating :

$R_j(t+1) = NewRating(\rho_d \rho_{ACF} a S_{f_k} + \sqrt{1- rho_d^2 \rho_{ACF}^2 a^2} \mathcal{N}(0,1) , R_{ini})$ p 39

Si il y a défaut alors $BigDebtor_j.Exposure(y,t+1) = 0$

Puis on ajuste $Gpool.Exposure(y,t+1)$, sachant que pendant la période révolving, $Total.Exposure(y,t) = BigDebtor_j.Exposure(y,t) + Gpool.Exposure(y,t)$ est constante

*The deals are reload so that total exposure stays flat p47*

- On test **[3eme condition]** : Trigger hit p46 :

le trigger est hit si :

$$ BRpoolLoss_k(y,t) > DefTriggerLimit_k \frac{LHR (GPool.Exposure(y,t) + \sum_j BigDebtor_j.Exposure(y,t))}{DSO_k/30} LGDd_k (1-DilRate_k)$$ 

avec $$ BRpoolLoss_k(y,t) = GPoolLoss_k(y,t) + BigDebtorLoss_k(y,t)$$ 


dont $$ GPoolLoss_k(y,t) =  GPool_k.Exposure(y,t) LHR \frac{1}{max(\frac{DSO_k}{30} - amt +1 ;1)} \mathcal{N}\left( \frac{\mathcal{N}^{-1}(GPool_kDefaultRate) - \sqrt{\rho_{def}} P_{f_k}(t)}{\sqrt{1-\rho_{def}}} \right) LGDd_k (1-DilRate_k) \ \ [CL]\ p62$$

ou $$ \rho_{def} = Correl_KMV(\mathcal{N}^{-1}(GPool_kDefaultRate) ; GPool_kStdDevRate ; \rho_D)$$

et $$BigDebtorLoss(y,t) = \sum_j BigDebtor_j.Exposure(y,t) LGDd_k (1-DilRate_k) \frac{1}{max(\frac{DSO_k}{30} - amt +1 ;1)} \ \ [CL]\ p62 $$

**Etape 4** : Si l'une des 3 conditions est atteinte, on passe à l'amortissement et les pertes subies sont la sommes des $BRpoolLoss_k(y,t)$ pendant 12 mois